In [2]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
from os.path import join
from sklearn import metrics
import os

In [3]:
local_dir = os.path.join("/DATA",os.environ.get("USER"),"MNIST_data")
os.makedirs(local_dir,mode=0o755, exist_ok=True)

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(local_dir, one_hot=True)

Extracting /DATA/solomiia.kurchaba/MNIST_data/train-images-idx3-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/t10k-labels-idx1-ubyte.gz


Parametry sieci

In [5]:
num_input = 28 
time_steps = 28 
hidden = 128
output_classes = 10
lr=0.0001
batch_size=128

Przygotowanie zbiorów treningowego (x_batches, y_batches) oraz testowego (x_test,y_test)

x-dane wejściowe, y-dane wyjściowe (identyfikator obrazka)

Dla tego żeby załadować dane wejściowe do sieci muszą być przygotowane w postaci trójwymiarowego tensora, gdzie pierwzy wymiar to wielkość zbioru, drugi-ilość kroków czasowych (ilość kroków z przeszłości, które pamięta sieć).Trzeci wymiar - ilosć zmiennych ładowanych do sieci w jednym kroku czasowym. Ponieważ mamy obrazki 28x28, przedstawiamy ich jako 28 zmiennych ładowanych w 28-miu krokach czasowych.

Dane wyjściowe zostawiamy tak jak są, czyli dwuwymiarowy tensor, gdzie pierwszy wymiar wielkość zbioru, drugi-liczba zmiennych.




In [7]:
x_tr, y_batches = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
x_batches = x_tr.reshape(-1, time_steps, num_input)
x_t=mnist.test.images
x_test=x_t.reshape(-1,time_steps,num_input)
y_test=mnist.test.labels

In [240]:
print(np.shape(x_b),np.shape(x_batches),np.shape(y_batches))
print(np.shape(x_t),np.shape(x_test))
print(np.shape(y_test))

(128, 784) (128, 28, 28) (128, 10)
(10000, 784) (10000, 28, 28)
(10000, 10)


# Konstrukcja sieci

1. Rozmiary Placecholderów dla danych wejściowych i wyjściowych muszą być zgodne z wymiarem przygotowanych danych;
2. Ponieważ korzystamy ze statycznej wersji sieci musimy zgóry podać ile kroków czasowych zamierzamy robić. Dlatego dane wejściowe muszą być jeszcze raz transformowane.
3. Po funkcji unstack mamy 28 tensorów (bo mamy 28 kroków czasowych) o wymiarze (rozmiar zbioru, ilość zmiennych)
4. Zamiast LSTM korzystam z RNN, tak jak ma taką samą zasadę działania, ale posiada dużo mniej parametrów wewnętrznych-proces uczenia się przebiega szybciej.


In [8]:
from tensorflow.contrib import rnn
tf.reset_default_graph()
X=tf.placeholder(tf.float32,[None,time_steps,num_input])
Y=tf.placeholder(tf.float32,[None,output_classes])
x=tf.unstack(X,time_steps,1)
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden) # here the first set of weights and biases is defined
rnn_output, states=rnn.static_rnn(basic_cell,x,dtype=tf.float32)
stacked_rnn_output=tf.reshape(rnn_output[-1],[-1,hidden])
stacked_outputs=tf.layers.dense(stacked_rnn_output,output_classes)# the output layer. (+additional set of weights and biases)
predict=tf.nn.softmax(stacked_outputs)
y_pred=tf.argmax(predict,1)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=Y))

optimizer=tf.train.AdamOptimizer(learning_rate=lr)
training_op=optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init=tf.global_variables_initializer()


# Aktywacja sieci
Ładujemy dane treningowe i testowe. Każde 100 iteracji treningowe wyświetlana wartość entropii.

Procent uzyskanych dobrych odpowiedzi podczas ostatniej iteracji traningowej: 99.22%

Procent uzyskanych poprawnych odpowiedzi na zbiorze testowym: 61.82%

Widzimy że rekurencyjne sieć na takiego typu danych działa dość kiepsko.



In [11]:
epochs=1000
with  tf.Session() as sess:
    init.run()
    for ep in range(epochs):
        sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
        loss=loss_op.eval(feed_dict={X: x_batches, Y: y_batches})
        if ep % 100 == 0:
            loss=loss_op.eval(feed_dict={X: x_batches, Y: y_batches})
            print(ep,"\tloss:",loss)
    train_accuracy=accuracy.eval(feed_dict={X: x_batches, Y: y_batches})  
    print(train_accuracy)
    test_accuracy=accuracy.eval(feed_dict={X:x_test, Y:y_test}) 
    print(test_accuracy)
        
        
sess.close()        
            
  

        

0 	loss: 2.29335
100 	loss: 1.90679
200 	loss: 1.662
300 	loss: 1.55047
400 	loss: 1.5141
500 	loss: 1.4886
600 	loss: 1.4842
700 	loss: 1.48219
800 	loss: 1.4809
900 	loss: 1.47369
0.992188
0.6182
